# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
city_df=pd.read_csv("../output_data/cities.csv")
city_df.head()


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
locations=city_df[["Lat", "Lng"]]

rating = city_df["Humidity"].astype(float)

In [5]:
# Plot Heatmap
fig = gmaps.figure()


# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=5,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
city_df=city_df.loc[(city_df["Max Temp"]>59)]
city_df.count()

City_ID       344
City          344
Cloudiness    344
Country       341
Date          344
Humidity      344
Lat           344
Lng           344
Max Temp      344
Wind Speed    344
dtype: int64

In [7]:
city_df=city_df.loc[(city_df["Max Temp"]<86)]
city_df.count()

City_ID       274
City          274
Cloudiness    274
Country       271
Date          274
Humidity      274
Lat           274
Lng           274
Max Temp      274
Wind Speed    274
dtype: int64

In [8]:
city_df=city_df.loc[(city_df["Humidity"]<30)]
city_df.count()

City_ID       13
City          13
Cloudiness    13
Country       11
Date          13
Humidity      13
Lat           13
Lng           13
Max Temp      13
Wind Speed    13
dtype: int64

In [9]:
city_df.head(15)

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
7,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44
14,14,constitucion,5,MX,1558378757,4,23.99,-104.67,84.20,16.11
30,30,gobabis,0,NaN,1558378567,24,-22.45,18.97,68.00,2.24
52,52,taybad,78,IR,1558378763,18,34.74,60.78,80.16,4.88
167,167,yumen,2,CN,1558378785,10,40.29,97.04,59.10,1.50
230,230,moses lake,1,US,1558378804,22,47.13,-119.28,69.80,13.87
259,259,hami,0,CN,1558378834,11,42.84,93.51,69.72,6.20
282,282,slave lake,1,CA,1558378673,25,55.28,-114.77,62.60,17.22
299,299,jiuquan,0,CN,1558378844,14,39.73,98.49,59.64,5.50
324,324,sidney,1,US,1558378850,26,47.72,-104.16,61.00,14.99


In [10]:
clean_city=city_df.dropna(how="any")
clean_city.count()

City_ID       11
City          11
Cloudiness    11
Country       11
Date          11
Humidity      11
Lat           11
Lng           11
Max Temp      11
Wind Speed    11
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
hotel_df=clean_city
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
14,14,constitucion,5,MX,1558378757,4,23.99,-104.67,84.20,16.11
52,52,taybad,78,IR,1558378763,18,34.74,60.78,80.16,4.88
167,167,yumen,2,CN,1558378785,10,40.29,97.04,59.10,1.50
230,230,moses lake,1,US,1558378804,22,47.13,-119.28,69.80,13.87
259,259,hami,0,CN,1558378834,11,42.84,93.51,69.72,6.20
282,282,slave lake,1,CA,1558378673,25,55.28,-114.77,62.60,17.22
299,299,jiuquan,0,CN,1558378844,14,39.73,98.49,59.64,5.50
324,324,sidney,1,US,1558378850,26,47.72,-104.16,61.00,14.99
367,367,pampierstad,0,ZA,1558378859,23,-27.78,24.69,67.02,10.38
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17


In [12]:
hotel_df["Hotel Name"]=""
hotel_df

<ipython-input-12-4b7c85aad61d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"]=""


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
14,14,constitucion,5,MX,1558378757,4,23.99,-104.67,84.20,16.11,
52,52,taybad,78,IR,1558378763,18,34.74,60.78,80.16,4.88,
167,167,yumen,2,CN,1558378785,10,40.29,97.04,59.10,1.50,
230,230,moses lake,1,US,1558378804,22,47.13,-119.28,69.80,13.87,
259,259,hami,0,CN,1558378834,11,42.84,93.51,69.72,6.20,
282,282,slave lake,1,CA,1558378673,25,55.28,-114.77,62.60,17.22,
299,299,jiuquan,0,CN,1558378844,14,39.73,98.49,59.64,5.50,
324,324,sidney,1,US,1558378850,26,47.72,-104.16,61.00,14.99,
367,367,pampierstad,0,ZA,1558378859,23,-27.78,24.69,67.02,10.38,
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17,


In [13]:
cities=hotel_df["City"].to_list()
cities

['constitucion',
 'taybad',
 'yumen',
 'moses lake',
 'hami',
 'slave lake',
 'jiuquan',
 'sidney',
 'pampierstad',
 'birjand',
 'paraiso']

In [26]:
# geocoordinates
cities=hotel_df["City"].values
for target_city in cities:
    target_url = f'https://maps.googleapis.com/maps/api/geocode/json?address={target_city}&key={g_key}'
    geo_data = requests.get(target_url).json()
    lat = geo_data["results"][0]["geometry"]["location"]["lat"]
    lng = geo_data["results"][0]["geometry"]["location"]["lng"]
    target_coordinates = f"{lat}, {lng}"
    target_radius = 5000
    target_type = "lodging"

    # set up a parameters dictionary
    params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    #     print(response.text)
    if response.status_code == 200:
        places_data=response.json()
#         print(json.dumps(places_data, indent=4, sort_keys=True))
        print(places_data["results"][0]["name"])
        
        

Constitucion, MX
دفتردستک محمدسیاه
玉门宾馆
Ramada by Wyndham Moses Lake
Post And Telecommunication Hotel
Super 8 by Wyndham Slave Lake AB
Jiuquan Hotel
Hampton Inn Sidney
Bonita Park
Birjand Tourist Hotel
Englewood Beach & Yacht Club


In [27]:
# geocoordinates
def get_hotel_names(target_city):
    target_url = f'https://maps.googleapis.com/maps/api/geocode/json?address={target_city}&key={g_key}'
    geo_data = requests.get(target_url).json()
    lat = geo_data["results"][0]["geometry"]["location"]["lat"]
    lng = geo_data["results"][0]["geometry"]["location"]["lng"]
    target_coordinates = f"{lat}, {lng}"
    target_radius = 5000
    target_type = "lodging"

    # set up a parameters dictionary
    params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    #     print(response.text)
    if response.status_code == 200:
        places_data=response.json()
#         print(json.dumps(places_data, indent=4, sort_keys=True))
        return (places_data["results"][0]["name"])
    else:
        return ""

In [34]:
for item in hotel_df.itertuples():
    hotel_df.loc[item.Index,"Hotel Name"]=get_hotel_names(item.City)

C:\Users\cdtn8\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [35]:
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
14,14,constitucion,5,MX,1558378757,4,23.99,-104.67,84.20,16.11,"Constitucion, MX"
52,52,taybad,78,IR,1558378763,18,34.74,60.78,80.16,4.88,دفتردستک محمدسیاه
167,167,yumen,2,CN,1558378785,10,40.29,97.04,59.10,1.50,玉门宾馆
230,230,moses lake,1,US,1558378804,22,47.13,-119.28,69.80,13.87,Ramada by Wyndham Moses Lake
259,259,hami,0,CN,1558378834,11,42.84,93.51,69.72,6.20,Post And Telecommunication Hotel
282,282,slave lake,1,CA,1558378673,25,55.28,-114.77,62.60,17.22,Super 8 by Wyndham Slave Lake AB
299,299,jiuquan,0,CN,1558378844,14,39.73,98.49,59.64,5.50,Jiuquan Hotel
324,324,sidney,1,US,1558378850,26,47.72,-104.16,61.00,14.99,Hampton Inn Sidney
367,367,pampierstad,0,ZA,1558378859,23,-27.78,24.69,67.02,10.38,Bonita Park
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17,Birjand Tourist Hotel


In [36]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [38]:
# Add marker layer ontop of heat map
# Create bank symbol layer
marker_layer = gmaps.marker_layer(
    locations, info_box_content=hotel_info)


# Display figure
fig = gmaps.figure()
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))